In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.crs as ccrs
from netCDF4 import Dataset
from shapely.geometry.polygon import LinearRing

import Utils, pygplates
from parameters import parameters

#
# The central point of the Orthographic projection
#
central_lon=-40.0
central_lat=-20.0

draw_velocity_vectors = True

rotation_files = Utils.get_files(parameters['rotation_files'])
topology_files = Utils.get_files(parameters["topology_files"])

agegrid_cmap = Utils.get_age_grid_color_map_from_cpt('agegrid.cpt')

for time in range(0, 231, 5):
    agegrid_file = Utils.download_agegrid(time)
    #reconstruct coastlines and topology
    print(f"reconstructing at {time} Ma...")

    resolved_topologies = []
    shared_boundary_sections = []
    #use pygplates to resolve the topologies
    pygplates.resolve_topologies(topology_files, rotation_files, resolved_topologies, time, 
                                 shared_boundary_sections)

    #reconstruct ore deposits
    reconstructed_deposits = []
    pygplates.reconstruct(
                    '../data/PorCuEX2008.gpmlz', 
                    rotation_files, 
                    reconstructed_deposits, 
                    time, 0)

    #coastlines
    reconstructed_geometries = []
    pygplates.reconstruct(
                    parameters['coastlines'], 
                    rotation_files, 
                    reconstructed_geometries, 
                    time, 0)

    #subduction zones
    subduction_geoms=[]
    Utils.get_subduction_geometries(subduction_geoms, shared_boundary_sections)

    #velocity vectors
    x,y, u,v = Utils.get_velocity_x_y_u_v(time,pygplates.RotationModel(rotation_files),topology_files)

    # plot the map
    fig = plt.figure(figsize=(12,8),dpi=96)
    ax = plt.axes(projection=ccrs.Orthographic(central_longitude=central_lon, central_latitude=central_lat))
    ax.gridlines()

    if agegrid_file:
        img = Dataset(agegrid_file) #age grid
        cb=ax.imshow(img.variables['z'], origin='lower', transform=ccrs.PlateCarree(),
              extent=[-180, 180, -90, 90], vmax=230, vmin=0, cmap=agegrid_cmap)

    #plot coastlines
    for geom in reconstructed_geometries:
        lat, lon =zip(*(geom.get_reconstructed_geometry().to_lat_lon_list()))
        plt.plot(lon, lat,
             color='black', linewidth=.5, #the coastlines in black
             transform=ccrs.Geodetic(),
        )
        
    #plot deposits
    deposit_lons=[]
    deposit_lats=[]
    deposit_ages=[]
    for geom in reconstructed_deposits:
        lat, lon = geom.get_reconstructed_geometry().to_lat_lon()
        deposit_lons.append(lon)
        deposit_lats.append(lat)
        begin_time, end_time = geom.get_feature().get_valid_time()
        deposit_ages.append(begin_time)
    ax.scatter(deposit_lons, deposit_lats, 50, marker='.',c=deposit_ages,  
               cmap=agegrid_cmap, vmax=230, vmin=0, transform=ccrs.PlateCarree())

    #plot topological plates boundaries
    for t in resolved_topologies:
        lat, lon =zip(*(t.get_resolved_boundary().to_lat_lon_list()))
        plt.plot(lon, lat,
             color='blue', linewidth=.5, #the topological plates boundaries in blue
             transform=ccrs.Geodetic(),
        )

    #plot subduction zones
    for geom, aspect in subduction_geoms:
        lat, lon =zip(*(geom.to_lat_lon_list()))
        plt.plot(lon, lat,
             color='blue', linewidth=1.5, #the subduction zones in blue
             transform=ccrs.Geodetic(),
        )
        teeth = Utils.get_subduction_teeth(lon, lat, triangle_aspect=aspect)
        for tooth in teeth:
            ring = LinearRing(tooth)
            ax.add_geometries([ring], ccrs.PlateCarree(), facecolor='b', edgecolor='black', alpha=1)


    if draw_velocity_vectors:
        #draw the velocity vectors
        #Some arrows are long and some are very short. To make the plot clearer, we nomalize the velocity magnitude.
        #And use color to denote the different speed.
        u = np.array(u)
        v = np.array(v)
        mag = np.sqrt(u*u+v*v)
        u = u/mag
        v = v/mag
        ax.quiver(x, y, u, v, mag,transform=ccrs.PlateCarree(),cmap='jet')    

    plt.title(f'{time} Ma')
    if agegrid_file:
       fig.colorbar(cb, shrink=0.5, label='Age(Ma)', orientation="horizontal", pad=0.05)

    plt.savefig(Utils.get_tmp_dir() + f'ortho_{time}_Ma.png',bbox_inches='tight',pad_inches=0.1)
    plt.close()
    print(f'plotting {time}Ma')
   


reconstructing at 0 Ma...


/home/mchin/anaconda3/envs/machine_learning/lib/python3.7/site-packages/ipykernel_launcher.py:123: RuntimeWarning: invalid value encountered in true_divide
/home/mchin/anaconda3/envs/machine_learning/lib/python3.7/site-packages/ipykernel_launcher.py:124: RuntimeWarning: invalid value encountered in true_divide
/home/mchin/anaconda3/envs/machine_learning/lib/python3.7/site-packages/cartopy/mpl/geoaxes.py:1831: RuntimeWarning: invalid value encountered in less
  u, v = self.projection.transform_vectors(t, x, y, u, v)
/home/mchin/anaconda3/envs/machine_learning/lib/python3.7/site-packages/cartopy/mpl/geoaxes.py:1831: RuntimeWarning: invalid value encountered in greater
  u, v = self.projection.transform_vectors(t, x, y, u, v)
/home/mchin/anaconda3/envs/machine_learning/lib/python3.7/site-packages/cartopy/mpl/geoaxes.py:1831: UserWarning: Some vectors at source domain corners may not have been transformed correctly
  u, v = self.projection.transform_vectors(t, x, y, u, v)
/home/mchin/anaco

KeyboardInterrupt: 

Error in callback <function flush_figures at 0x7f5c3c773170> (for post_execute):


KeyboardInterrupt: 

In [4]:
%%capture --no-stdout

import moviepy.editor as mpy

frame_list = [Utils.get_tmp_dir() + f'ortho_{time}_Ma.png' for time in range(230, -1, -5)]
clip = mpy.ImageSequenceClip(frame_list, fps=2)
clip.write_videofile( Utils.get_tmp_dir() + "ortho_south_america.mp4")
print('video has been created!')

Moviepy - Building video test-case-south-america-lala/tmp/ortho_south_america.mp4.
Moviepy - Writing video test-case-south-america-lala/tmp/ortho_south_america.mp4

Moviepy - Done !
Moviepy - video ready test-case-south-america-lala/tmp/ortho_south_america.mp4
video has been created!


In [5]:
from IPython.display import Video

Video(Utils.get_tmp_dir() + "ortho_south_america.mp4",width=700)